In [3]:
#Title: 
#What is the sex of an individual who is diagnosed with heart disease predicted by …. Symptoms?


#Intro:
#-describe heart disease
#-background info
#-gender difference


#Preliminary exploratory data analysis:


#Methods:


#Expected Outcomes and Predicitions 
